# Check Device (RFID Reader)

This module displays RFID device parameter.

Module interaction:

    Environement selection
    Premise selection
    Device selection in the premise


## Select an environment

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtil

Select the Premise where is located the device.

In [ ]:
#%%
status = True
myEnv = chooseEnv.choosePremise()

In [ ]:
#%% display choice
if not myEnv.get('PremiseCode'):
    text = 'No Premise Found '
    uiUtils.displayMessage('Device search',text)
    print(text)
    status = False
else:  
    print("Organization: " + myEnv.get('urlDP') + " Premise code: "+  myEnv.get('PremiseCode'))

Load all the devices linked to the premise

In [ ]:
if status:
    listOfDevices = deviceUtils.getDevicesOfPremise(myEnv)
    if not listOfDevices:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' No device found '
        uiUtils.displayMessage('Devices search',text)
        print(text)
        status = False
        
if status:       
    listOfNames = []
    for device in listOfDevices:
        nameOfDevice = device.get('name')
        if not nameOfDevice:
            nameOfDevice = "NoName"
        idOfDevice = device.get('id')
        str2Display =  idOfDevice + ":" +nameOfDevice
        listOfNames.append(str2Display)

In [ ]:
listOfNames

### Display the list for selection

In [ ]:
if status:
    deviceC = uiUtils.chooseWithMenu('Devices',listOfNames)
    if not deviceC:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' No Device Selected '
        uiUtils.displayMessage('Devices Selection',text)
        print(text)
        status = False
    else:
        tmp = deviceC.split(':')
        deviceId = tmp[0]
        if not deviceId:
            text = "Premise: " + str(myEnv.get('PremiseCode')) + ' Error in Device selection: '+deviceC
            uiUtils.displayMessage('Devices Selection',text)
            print(text)
            status = False  

### Request device definition

In [ ]:
#%% device information
if status:
    deviceDefinition = deviceUtils.getDeviceById(myEnv,deviceId)
    if not deviceDefinition:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' Error getting device information: '+ deviceId
        uiUtils.displayMessage('Devices Selection',text)
        print(text)
        status = False        

In [ ]:
#%% pov (antennas) information
if status:
    listOfPov = deviceDefinition.get('devicePov')
    if not listOfPov:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' Error getting device PointOfView information: '+ deviceId
        uiUtils.displayMessage('Devices Selection',text)
        print(text)
        status = False          
    else:
        povDetails = {}
        for pov in listOfPov:
            povid = pov.get('id')
            povDetails[povid] = povUtils.getPovById(myEnv,povid)

### Display device definition

In [ ]:
if status:
    text = "Premise: " + str(myEnv.get('PremiseCode'))
    device = deviceDefinition.get('device')
    text  = text +'\n' + 'device ID: ' +  deviceId
    text  = text +'\n' + 'device IP: ' +  device.get('address')
    text  = text +'\n' + 'device NAME: ' +  device.get('name')

    dType = deviceDefinition.get('deviceType')
    typeText = "Brand " +dType.get('brand')+ " Model "+dType.get('name')+" Version " +dType.get('version')
    text  = text +'\n' + 'device type : ' +  typeText

    text  = text +'\n' + 'device mode : ' 
    deviceModes = deviceDefinition.get('deviceMode')
    for mode in deviceModes:
        text  = text +'\n' + '    '+ str(mode.get('name'))


    deviceStarUpMode = deviceDefinition.get('deviceStarUpMode') 
    text  = text +'\n' + 'startUp mode : ' + deviceStarUpMode.get('name')

    text  = text +'\n' + 'POV : ' 

    for povid in povDetails:
        povDetail = povDetails[povid]
        text  = text +'\n' + '    ID: '+ str(povid)
        povGeneralDefinition = povDetail.get('pov')
        text  = text +'\n' + '        name: '+ str(povGeneralDefinition.get('name'))
        text  = text +'\n' + '        address: '+ str(povGeneralDefinition.get('address'))
        text  = text +'\n' + '        isEnable: '+ str(povGeneralDefinition.get('isEnabled'))

        povType = povDetail.get('povType')
        if povType:
            typeText = "Brand " +povType.get('brand')+ " Model "+povType.get('name')+" Version " +povType.get('version')
            text  = text +'\n' + '        type: '+ typeText
        else:
            text  = text +'\n' + '        type: No PoV Type'
            
        povFixture = povDetail.get('povFixture')
        if povFixture:
            text  = text +'\n' + '        fixture id: '+ povFixture.get('id')
            text  = text +'\n' + '        fixture name: '+ povFixture.get('name')
        else:
            text  = text +'\n' + '        fixture id: No fixture found'
            
        povSettings = povDetail.get('povSettings')


    text  = text +'\n' + 'device Fixture: ' +  str(deviceDefinition.get('fixtureId'))


    uiUtils.displayMessage('Device parameters',text)